In [1]:
# import os
# import sys
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

import plotly.express as px
import plotly.graph_objects as go


import pandas as pd
import numpy as np
from quantfreedom.utils.caching import clear_cache
from quantfreedom.nb.simulate import backtest_df_array_only
from quantfreedom.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)
from quantfreedom.indicators.talib_ind import from_talib
from quantfreedom.evaluators.evaluators import eval_is_below

# np.set_printoptions(formatter={'float_kind':"{:.2f}".format})

# pd.options.display.float_format = '{:,.2f}'.format

prices = pd.read_csv(
    '../data/30min.csv', index_col='time')

rsi_ind = from_talib(
    func_name='rsi',
    df_prices=prices,
    cart_product=False,
    combos=False,
    timeperiod=[15,30,45],
)

rsi_eval = eval_is_below(
    rsi_ind,
    np.arange(40, 61, 10),
)

In [2]:
strat_array, settings_array = backtest_df_array_only(
    open_prices=prices.open.values,
    high_prices=prices.high.values,
    low_prices=prices.low.values,
    close_prices=prices.close.values,
    entries=rsi_eval.values,
    equity=1000.,
    fee_pct=.06,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=np.arange(1,6.1,1),
    risk_rewards=np.arange(1,10.1,1),
    size_pct=1.,
    gains_pct_filter=100,
    # sl_pcts=np.arange(1,6,1),
    tsl_true_or_false=True,
    tsl_pcts_init=np.arange(1,6.1,1),
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_trail_by_pct=np.arange(1,5.1,1),
    tsl_when_pct_from_avg_entry=np.arange(1,5.1,1),
)

In [3]:
pd.DataFrame(strat_array).sort_values(by='to_the_upside', ascending=False).head(10)

,or_set,ind_set,total_trades,gains_pct,win_rate,to_the_upside,total_pnl,ending_eq
1440,3448,0,43.00,105.54,53.49,0.73,"1,055.41","2,055.41"
1433,3443,8,60.00,100.13,56.67,0.73,"1,001.31","2,001.31"
1428,3441,4,65.00,112.85,50.77,0.72,"1,128.51","2,128.50"
10985,6276,8,109.00,152.11,72.48,0.72,"1,521.09","2,521.09"
5960,5043,5,81.00,237.55,50.62,0.72,"2,375.51","3,375.51"
1430,3442,5,65.00,101.15,53.85,0.71,"1,011.49","2,011.49"
11368,6400,8,165.00,199.57,73.33,0.71,"1,995.66","2,995.66"
5962,5043,8,81.00,246.21,50.62,0.70,"2,462.10","3,462.10"
5968,5044,5,72.00,228.25,47.22,0.70,"2,282.55","3,282.55"
17988,7776,8,108.00,185.97,72.22,0.70,"1,859.66","2,859.66"


In [4]:
df_cart = pd.DataFrame(settings_array).dropna(axis='columns', thresh=1)
for i in range(len(SL_BE_or_Trail_BasedOn._fields)):
    df_cart.replace({'tsl_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
    df_cart.replace({'sl_to_be_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
df_cart = df_cart.T

In [5]:
df_cart

,0,1,2,3,4,5,6,7,8,9,...,24992,24993,24994,24995,24996,24997,24998,24999,25000,25001
ind_set_id,5,5,5,5,5,8,5,8,5,8,...,5,7,8,0,1,2,4,5,7,8
max_equity_risk_pct,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00
risk_rewards,4.00,4.00,4.00,4.00,4.00,4.00,5.00,5.00,5.00,5.00,...,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00
size_pct,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
tsl_based_on,low_price,low_price,low_price,low_price,low_price,low_price,low_price,low_price,low_price,low_price,...,low_price,low_price,low_price,low_price,low_price,low_price,low_price,low_price,low_price,low_price
tsl_pcts_init,2.00,2.00,2.00,2.00,2.00,3.00,2.00,2.00,2.00,2.00,...,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00
tsl_trail_by_pct,5.00,5.00,5.00,5.00,5.00,1.00,1.00,1.00,1.00,1.00,...,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00
tsl_when_pct_from_avg_entry,1.00,2.00,3.00,4.00,5.00,5.00,3.00,3.00,4.00,4.00,...,4.00,4.00,4.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00


In [6]:
df_cart[5960]

ind_set_id                             5
max_equity_risk_pct                 4.00
risk_rewards                        4.00
size_pct                            1.00
tsl_based_on                   low_price
tsl_pcts_init                       4.00
tsl_trail_by_pct                    4.00
tsl_when_pct_from_avg_entry         4.00
Name: 5960, dtype: object

In [9]:
rsi_eval.iloc[:, [5]]

RSI_timeperiod,30
RSI_is_below,60
time,
2020-10-01 00:00:00,False
2020-10-01 00:30:00,False
2020-10-01 01:00:00,False
2020-10-01 01:30:00,False
2020-10-01 02:00:00,False
...,...
2021-02-22 21:30:00,True
2021-02-22 22:00:00,True
